# Week 6 - SMM695

Matteo Devigili

June, 28th 2022

[_PySpark_](https://spark.apache.org/docs/latest/api/python/index.html#): during this lecture, we will approach Spark through Python

<img src="images/_1.png" width="20%">

**Agenda**:
1. Introduction to Spark
1. Installing PySpark
1. PySpark Basics
1. PySpark and Pandas
1. PySpark and SQL
1. Load data from your DBMS

# Introduction to Spark

**Big Data Challenge**:

* Cost of storing data has dropped
* The need for parallel computation has increased

![IBM Blue Gene\L](https://www.ibm.com/ibm/history/ibm100/images/icp/U225116Q82800V30/us__en_us__ibm100__blue_gene__man_next_blue_gene__620x350.jpg)

**Note**: [IBM Blue Gen\L](https://www.ibm.com/ibm/history/ibm100/us/en/icons/bluegene/)

**What is [Apache Spark](https://spark.apache.org)**?

> "Apache Spark is a unified computing engine and a set of libraries for parallel data processing on computer clusters"

[Chambers and Zaharia 2018](#references)

**Programming Languages Supported**:

<img src="images/_0.png" width="50%">

**Spark's philosophy**:

* *Unified*: Spark offers a large variety of data analytics tools
* *Computing Engine*: Spark focuses on computing, not on storage
* *Libraries*: Spark has different libraries to perform several tasks

**Apache Spark Libraries**:

* *Spark SQL*
* *Spark Streaming*
* *Spark MLlib*
* *Spark GraphX*

[Third-party projects](https://spark.apache.org/third-party-projects.html)

**Spark Application**:

| Component ||Role |
|----|----|---|
| *Spark Driver*| | Execute user-defined tasks |
| *Cluster Manager* | | Manage workers nodes|
| *Executors* | | Execute tasks |

<img src="images/_5.png" width=80%>

**From Python to Spark code and back**:

![The relationship between the SparkSession and Spark’s Language API
](https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/assets/spdg_0202.png)

Source: _Bill Chambers, Matei Zaharia 2018_ (p. 23)

# Installing PySpark

There are several ways to set-up PySpark on your local machine. Here, two methods are discussed:
* Pure-python users: 
```python
pip install pyspark
```
* Conda users:
```python
conda install pyspark
```
Further info at [Spark Download page](https://spark.apache.org/downloads.html).

## Requirements

Pay attention to the following:

>Spark runs on Java 8/11

Check java version running on your machine. Type the following on your terminal:
```python
java -version
```

If you are running a different Java version, install java 8/11! Check out [Spark Downloading info](https://spark.apache.org/docs/latest/#downloading).

# PySpark - Basics

## Libraries

In [ ]:
#to create a spark session object
from pyspark.sql import SparkSession

# functions
import pyspark.sql.functions as F

# data types
from pyspark.sql.types import *

# import datetime 
from datetime import date as dt

* More info on **Functions** at these [link-1](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) & [link-2](https://spark.apache.org/docs/2.3.0/api/sql/index.html#year)
* More info on **Data Types** at this [link](https://spark.apache.org/docs/latest/sql-ref-datatypes.html)

## Opening a Session

The **SparkSession** is a driver process that enables:

* to control our Spark Application
* to execute user-defined manipulations

Check this [link](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession) for further reference.

In [ ]:
# to open a Session
spark = SparkSession.builder.appName('last_dance').getOrCreate()

**Spark UI**

<img src="images/_6.png" width=60%>

The spark UI is useful to monitor your application. You have the following tabs:

* *Jobs*: info concerning Spark jobs
* *Stages*: info on individual stages and their tasks
* *Storage*: info on data that is currently in our spark application
* *Environment*: info on configurations and current settings of our application
* *Executors*: info on the executors that run our application
* *SQL*: refers to both SQL and DataFrames

In [ ]:
spark

## Create Dataframe

In order to create a dataframe from scratch, we need to:
1. Create a schema, passing:
  * Column names
  * Data types
1. Pass values as an array of tuples

In [ ]:
# Here, I define a schema
# .add(field, data_type=None, nullable=True, metadata=None)

schema = StructType().add("id", "integer", True).add("first_name", "string", True).add(
    "last_name", "string", True).add("dob", "date", True)

'''
schema = StructType().add("id", IntegerType(), True).add("first_name", StringType(), True).add(
    "last_name", StringType(), True).add("dob", DateType(), True)
'''

# Then, I can pass some values
df = spark.createDataFrame([(1, 'Michael', "Jordan", dt(1963, 2, 17)),
                            (2, 'Scottie', "Pippen", dt(1965, 9, 25)),
                            (3, 'Dennis', "Rodman", dt(1961, 5, 16))],
                           schema=schema)

# Let's explore Schema structure
df.printSchema()

In [ ]:
# We can also leverage on functions to create a new column
df=df.withColumn('age', F.year(F.current_date()) - F.year(df.dob))

df.show()

**Transformations**

* Immutability: once created, data structures can not be changed
* Lazy evaluation: computational instructions will be executed at the very last

**Actions**

* view data
* collect data
* write to output data sources

# PySpark and Pandas

## Load a csv

Loading a csv file from you computer, you need to type:
* Pands:
  * db = pd.read_csv('path/to/movies.csv')
* Pyspark:
  * df = spark.read.csv('path/to/movies.csv', header=True, inferSchema=True)

Here, we will import a csv directly from GitHub. Data are provided by [FiveThirtyEight](https://github.com/fivethirtyeight)

[<img src="images/_2.png" width="50%">](https://fivethirtyeight.com/features/the-dollar-and-cents-case-against-hollywoods-exclusion-of-women/)

In [ ]:
# import pandas
import pandas as pd

# import SparkFiles
from pyspark import SparkFiles

# target dataset
url = 'https://raw.githubusercontent.com/fivethirtyeight/data/master/bechdel/movies.csv'

In [ ]:
# loading data with pandas
db = pd.read_csv(url)

# loading data with pyspark
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('movies.csv'), header=True, inferSchema=True)

## Inspecting dataframes

In [ ]:
# pandas info
db.info()

In [ ]:
# pyspark schema
df.printSchema()

In [ ]:
# pandas fetch 5
db.head(5)

In [ ]:
# pyspark fetch 5
df.show(5)

df.take(5)

In [ ]:
# pandas filtering:
db[db.year == 1970]

In [ ]:
# pyspark filtering:
df[df.year == 1970].show()

In [ ]:
# get columns and data types
print("""
Pandas db.columns:
===================
{}

PySpark df.columns:
===================
{}

Pandas db.dtype:
===================
{}

PySpark df.dtypes:
===================
{}

""".format(db.columns, df.columns, db.dtypes, df.dtypes), flush = True)

## Columns

In [ ]:
# pandas add a column
db['newcol'] = db.domgross/db.intgross

# pyspark add a column
df=df.withColumn('newcol', df.domgross/df.intgross)

In [ ]:
# pandas rename columns
db.rename(columns={'newcol': 'dgs/igs'}, inplace=True)

# pyspark rename columns
df=df.withColumnRenamed('newcol', 'dgs/igs')

## Drop

In [ ]:
# pandas drop `code' column
db.drop('code', axis=1, inplace=True)

# pyspark drop `code' column
df=df.drop('code')

In [ ]:
# pandas dropna()
db.dropna(subset=['domgross'], inplace=True)

# pyspark dropna()
df=df.dropna(subset='domgross')

## Stats

In [ ]:
# pandas describe
db.describe()

In [ ]:
# pyspark describe
df.describe(['year', 'budget']).show()

# Pyspark and SQL

In [ ]:
# pyspark rename 'budget_2013$'
df=df.withColumnRenamed('budget_2013$', 'budget_2013')

In [ ]:
# Create a temporary table 
df.createOrReplaceTempView('bechdel')

# Run a simple SQL command
sql = spark.sql("""SELECT imdb, year, title, budget FROM bechdel LIMIT(5)""")
sql.show()

In [ ]:
# AVG budget differences
sql_avg = spark.sql(
    """
    SELECT 
    binary, 
    COUNT(*) AS count, 
    format_number(AVG(budget),2) AS avg_budget, 
    format_number((SELECT AVG(budget) FROM bechdel),2) AS avg_budget_samp,
    format_number(AVG(budget_2013),2) AS avg_budget2013,
    format_number((SELECT AVG(budget_2013) FROM bechdel),2) AS avg_budget2013_samp
    FROM bechdel GROUP BY binary
    """
)

sql_avg.show()

# Load data from DBMS

To run the following you need to restart the notebook.

In [ ]:
# to create a spark session object
from pyspark.sql import SparkSession

## PostgreSQL

To interact with postgre you need to:
    
* Download the *postgresql-42.4.0.jar file* [here](https://jdbc.postgresql.org/download.html)
* Include the path to the downloaded jar file into SparkSession()

In [ ]:
# Open a session running data from PostgreSQL
spark_postgre = SparkSession \
    .builder \
    .appName("last_dance_postgre") \
    .config("spark.jars", "/Users/matteodevigili/GitHub/dms-smm695/.venv/share/py4j/postgresql-42.4.0.jar") \
    .getOrCreate()

In [ ]:
spark_postgre

In [ ]:
# Read data from PostgreSQL running at localhost
df = spark_postgre.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/pagila") \
    .option("dbtable", "film") \
    .option("user", "postgres") \
    .option("password", "smm695") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

In [ ]:
# get some stats
df.describe(['release_year', 'rental_rate', 'rental_duration']).show()

In [ ]:
# Create a temporary table 
df.createOrReplaceTempView('film')

# Run a simple SQL command
sql = spark_postgre.sql("""SELECT title, release_year, length, rating FROM film LIMIT(1)""")
sql.show()

## MongoDB

For further reference check the [Python Guide provided by Mongo](https://docs.mongodb.com/spark-connector/current/python-api/) or the [website for the mongo-spark connector](https://spark-packages.org/package/mongodb/mongo-spark).

In [ ]:
# add path to Mongo
spark_mongo = SparkSession \
    .builder \
    .appName("last_dance_mongo") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/amazon.music") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/amazon.music") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

In [ ]:
spark_mongo

In [ ]:
# load data from MongoDB
df = spark_mongo.read.format("mongo").load()

df.printSchema()

In [ ]:
# get some stats
df.describe(['overall', 'unixReviewTime']).show()

In [ ]:
# Create a temporary table 
df.createOrReplaceTempView('music')

# Run a simple SQL command
sql = spark_mongo.sql("""SELECT asin, date, helpful, overall, unixReviewTime FROM music LIMIT(1)""")
sql.show()

# References

* Bill Chambers, Matei Zaharia 2018,["Spark: The Definitive Guide"](https://www.oreilly.com/library/view/spark-the-definitive/9781491912201/) 

<img src="images/_3.png" width="20%">

* Pramod Singh 2019, ["Learn PySpark: Build Python-based Machine Learning and Deep Learning Models
"](https://www.ibs.it/learn-pyspark-build-python-based-libro-inglese-pramod-singh/e/9781484249604) 

<img src="images/_4.png" width="18%">